In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Создание клиента

В этом примере мы будем использовать [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) для доступа к LLM.

`model` определен как `gpt-4o-mini`. Попробуйте изменить модель на другую, доступную на платформе GitHub Models, чтобы увидеть разные результаты.

Для быстрого теста мы просто запустим простой запрос — `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Определение функций

В этом примере мы предоставим агенту доступ к инструменту, который является функцией с перечнем доступных туристических направлений и их доступностью.

Можно представить, что это сценарий, в котором туристический агент, например, имеет доступ к базе данных путешествий.

По мере изучения этого примера не стесняйтесь пробовать определять новые функции и инструменты, которые агент сможет вызывать.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Определение инструмента функции  
Чтобы агент мог использовать `vacation_destinations` как `FunctionTool`, нам нужно определить его как таковой.  

Мы также предоставим описание инструмента, которое поможет агенту понять, для чего этот инструмент используется в контексте задачи, запрошенной пользователем.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Определение агента

Теперь мы можем создать агента в приведенном ниже коде. Мы определяем `system_message`, чтобы дать агенту инструкции о том, как помогать пользователям находить места для отдыха.

Мы также устанавливаем параметр `reflect_on_tool_use` в значение true. Это позволяет использовать LLM для обработки ответа инструмента и отправки ответа на естественном языке.

Вы можете установить параметр в значение false, чтобы увидеть разницу.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Запуск агента

Теперь мы можем запустить агента с начальным сообщением пользователя, в котором он просит организовать поездку в Токио.

Вы можете изменить пункт назначения, чтобы увидеть, как агент реагирует на доступность выбранного города.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
